# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
from openai.types.chat import ChatCompletionMessageParam

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
client = OpenAI()

In [3]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

def format_message(question: str) -> list[ChatCompletionMessageParam]:
    return [{"role": "system", "content": "you are an expert python programmer with at least 15 years of experience."},
            {"role": "user", "content": question}]

In [5]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

response = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=format_message(question),  
    stream=True
)

# display the response as it streams in
display_id = display(Markdown("**Answer:** "), display_id=True)
collected_chunks = []
collected_messages = []

for chunk in response:
    collected_chunks.append(chunk)  # save the event response
    chunk_message = chunk.choices[0].delta  # extract the message - USE DOT NOTATION
    collected_messages.append(chunk_message)  # save the message
   
    # Check if content exists using hasattr() or getattr()
    if hasattr(chunk_message, 'content') and chunk_message.content:
        update_display(Markdown("**Answer:** " + ''.join(
            getattr(m, 'content', '') for m in collected_messages)), display_id=display_id.display_id)

**Answer:** The provided code snippet utilizes Python's `yield from` statement along with a set comprehension to yield values from a collection of authors found in a list of book dictionaries.

Let's break down the code step-by-step:

1. **Books Collection**: The variable `books` is expected to be an iterable (commonly a list) containing dictionaries, where each dictionary represents a book.

2. **Set Comprehension**: The code includes the following set comprehension:
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   - This part iterates over each `book` in the `books` collection.
   - For each `book`, it retrieves the value associated with the key `"author"` using the `.get()` method. 
     - The `.get()` method returns the value for the specified key if it exists; if the key is not found, it returns `None`.
   - The `if book.get("author")` condition filters out any books that do not have an author or have no value (e.g., `None`).
   - The result of the set comprehension is a set that contains unique authors from the books. Using a set ensures that there are no duplicate authors in the result.

3. **Yielding Authors**: The `yield from` statement allows yielding all the values from the set as a generator. 
   - This means that when the generator created by this code is iterated over, it will yield each unique author found in the `books` list one by one.

In summary, this code snippet creates a generator that yields unique authors from a list of book dictionaries. It effectively filters out any books without an author and avoids duplicate entries since it uses a set.

### Why Use `yield from`?
- `yield from` is useful for delegating the yield of values from another iterable (in this case, the set of authors). It simplifies the generator function and makes it straightforward to yield values from nested generators or iterables directly. 

### Example Usage:
If `books` is:
```python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": "Author A"},
    {"title": "Book Four", "author": None},
    {"title": "Book Five"},
]
```

The code would yield:
- "Author A"
- "Author B"

You can use this generator in a loop to receive each author:
```python
for author in unique_authors_generator:
    print(author)
``` 

Given this explanation, you should have a clear understanding of what this specific line of code does and its utility in Python.

In [6]:
# Get Llama 3.2 to answer (requires Ollama running locally)
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = ollama_client.chat.completions.create(
    model=MODEL_LLAMA,
    messages=format_message(question) 
)

display("\nLlama 3.2's answer:\n")
display(Markdown(response.choices[0].message.content))

"\nLlama 3.2's answer:\n"

This is a Python snippet that uses generator expressions to extract author names from a list of dictionaries called `books`.

Let's break it down:

* `{}` denotes an empty dictionary, but in this context, it's used as an expression. This syntax is called dictionary expression shorthand.
* `.get("author") for book in books if book.get("author")` defines a generator expression:
	+ `.get("author")` accesses the value associated with the key `"author"` in each dictionary (`book`). If the key doesn't exist, `.get()` defaults to returning `None`.
	+ `for book in books` iterates over the list of dictionaries (`books`) and assigns each dictionary to the variable `book`.
	+ `if book.get("author")` filters out any dictionaries where `"author"` is not present.
* `yield from ...`: This keyword tells Python to yield results from a generator expression. It's shorthand for creating a generator function with an explicit `yield` statement and another `yield from` inside.

So, putting it all together, this code generates values from a list of dictionaries:

```python
for author in (
    book.get("author") 
    # Only include books that have "author" as a key
    for book in books 
):
```

In other words, the expression `(book.get("author") for book in books if book.get("author"))` produces an iterator over author names. The `yield from` clause then forwards these values to the surrounding generator (not shown), which would iterate over them.

Here's a simple example using this code snippet:
```python
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2"}, 
    {"title": "Book 3", "author": "Jane Doe"}
]

for author in yield from ({book.get("author") for book in books if book.get("author")}):
    print(author)
```
Output:
```
John Doe
Jane Doe
```